In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

# Differential Rotation

Stars -- the Sun included -- can exhibit differential rotation, meaning the rotation period is different at different latitudes. Convection in the upper layers of a star can redistribute angular momentum, typically causing regions near the poles to rotate slower than the equator.

Differential rotation is usually modeled by a function of latitude that depends on a single parameter: the linear shear coefficient $\alpha$. Under this model, the angular velocity $\omega$ at a latitude $\phi$ on the surface is given by

$\omega(\phi) = \omega_{eq}(1 - \alpha \sin^2\phi)$,

where $\omega_{eq}$ is the angular velocity at the equator.

In `starry`, users can modify the `alpha` attribute of a `Map` instance to change the strength of differential rotation. The default is zero (i.e., solid body rotation). In this notebook, we'll show a few examples of this and test the accuracy and speed of our implementation.

But before we begin, there are two important caveats:

**1.** The implementation in `starry` is approximate. Under perfect differential rotation, features should twist and wind indefinitely as time goes on. It's impossible to capture this behavior with a spherical harmonic expansion of a fixed order, since the winding creates features on the surface on progressively tinier scales. After a certain amount of time, the `starry` approximation breaks down and things will look very weird. In order to keep things physical, there are two other parameters that users can tweak: the feature timescale `tau` and the lag `delta`. We'll explain these in more detail below, but the basic idea is to suppress the amplitude of features on the surface as they begin to wind. This is loosely based on reality: features like star spots will disappear on a certain timescale -- certainly long before they wind around the star.

**2.** The second major limitation of this implementation is that in reality, star spots might not shear *at all*. Magnetic forces should in principle keep small star spots from differentially rotating themselves. This means that star spots at different latitudes rotate at different rates, but the rotation rate is constant *within* a star spot. Unfortunately, there's no way of modeling this behavior non-parametrically with spherical harmonics, since there's no such thing as a starspot in `starry`. Rather, there's just a *process* (a *Gaussian* process, if you will) on the surface of the star that we can manipulate in a smooth and continuous way. So if you add a "spot" to a star (by calling `add_spot()`, for instance) and differentially rotate it, it *will* shear. That said, however, the giant spots we infer on the surfaces of other stars may actually be *groups* of dozens or even hundreds of tiny spots that we can't quite resolve. If that is the case, then it is quite possible that those groups will shear, and our method is actually fine at the resolution we're able to probe.

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np

starry.config.lazy = False

lats = [0, 30, 60]
lons = [0, 90, 180]
periods = [1.0, 0.75, 0.50]

time = np.linspace(0, 30, 1000)
flux = np.ones_like(time)

for lat, lon, per in zip(lats, lons, periods):
    spot = starry.Map(10, inc=60)
    spot.add_spot(lat=lat, lon=lon, sigma=0.1, intensity=-0.1)
    flux += spot.flux(theta=360.0 / per * time) - 1.0

plt.plot(time, flux)
plt.xlabel("time")
plt.ylabel("flux");

With these caveats in mind, let's dive in. 

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
from scipy.interpolate import interp1d

In [ ]:
starry.config.lazy = False
starry.config.quiet = True

## Validation of the method

First, we're going to show that the implementation in `starry` does what it's supposed to. Let's instantiate a degree `15` map of the Earth and differentially rotate it by numerically 
shifting each latitude by the correct amount. (This is just for fun; don't worry about the physics of why the Earth would differentially rotate!)

In [ ]:
map = starry.Map(15)
map.load("earth", sigma=0.1)
map.amp = 1.0
y = np.array(map.y)

### Numerical

In [ ]:
def diff_rotate(img, lat, lon, prot, alpha, t):
    img_rot = np.zeros_like(img)
    omega_eq = 360.0 / prot
    for i, lat_i in enumerate(lat):
        new_lon = lon[i] + omega_eq * alpha * t * np.sin(lat_i * np.pi / 180.0) ** 2
        new_lon = ((new_lon + 180) % 360) - 180
        func = interp1d(lon[i], img[i], fill_value="extrapolate")
        img_rot[i] = func(new_lon)
    return img_rot

Let's set $\alpha = 0.02$ and spin the Earth through ten rotations.

In [ ]:
# Rotation params
prot = 1.0
alpha = 0.02

# Get the image at t=0
res = 300
img = map.render(projection="rect", res=res)
lat, lon = map.get_latlon_grid(projection="rect", res=res)

# Get and plot the image at various times
img_rot = np.zeros((9, res, res))
fig, ax = plt.subplots(3, 3, figsize=(12, 6))
ax = ax.flatten()
for i, t in enumerate(np.linspace(0, 10, len(ax))):
    img_rot[i] = diff_rotate(img, lat, lon, prot, alpha, t)
    ax[i].imshow(img_rot[i], origin="lower", extent=(-180, 180, -90, 90), cmap="plasma")
    ax[i].set(xticks=[], yticks=[])
    ax[i].set_ylabel(r"${:.2f}$".format(t), fontsize=10)

Things look correct (as much as they're supposed to!): features at the poles are lagging features at the equator, by an amount equal to $\alpha \sin^2\phi$. (Note that we're in a frame co-rotating with the equator).

### Analytic

Let's now do the same thing, this time using the analytic method in `starry`.

Two important notes about this:

**1.** We're instantiating a degree `20` map, but only setting the coefficients up to degree `15`. This is therefore the exact same map as above, but we have five degrees of "padding". That's because, as we mentioned earlier, differential rotation *increases* the degree of a map over time. By giving us this extra padding, we can more accurately capture the shearing and twisting. This procedure -- instantiating a map of degree higher than you actually need -- is important when modeling differential rotation in `starry`.

In [ ]:
map = starry.Map(20)
map[1:16, :] = y[1:]
map.alpha = alpha
map.tau = np.inf

img_starry = np.zeros((9, res, res))
fig, ax = plt.subplots(3, 3, figsize=(12, 6))
ax = ax.flatten()
for i, t in enumerate(np.linspace(0, 10, len(ax))):
    img_starry[i] = map.render(projection="rect", res=res, theta=360.0 / prot * t)
    ax[i].imshow(
        img_starry[i], origin="lower", extent=(-180, 180, -90, 90), cmap="plasma"
    )
    ax[i].set(xticks=[], yticks=[])
    ax[i].set_ylabel(r"${:.2f}$".format(t), fontsize=10)

### Difference

In [ ]:
diff = img_rot - img_starry
vmin = min(np.min(diff), -np.max(diff))
vmax = -vmin

fig, ax = plt.subplots(3, 3, figsize=(12, 6))
ax = ax.flatten()
for i, t in enumerate(np.linspace(-10, 10, len(ax))):
    im = ax[i].imshow(
        diff[i],
        origin="lower",
        extent=(-180, 180, -90, 90),
        cmap="plasma",
        vmin=vmin,
        vmax=vmax,
    )
    ax[i].set(xticks=[], yticks=[])
    ax[i].set_ylabel(r"${:.2f}$".format(t), fontsize=10)
fig.colorbar(im, cax=fig.add_axes([0.92, 0.11, 0.025, 0.77]));

## Error analysis

In [ ]:
def get_error(ydeg=15, wta=30, **kwargs):

    # Instantiate
    map = starry.Map(ydeg, **kwargs)

    # Apply the differential rotation then undo it.
    # If the transform is one-to-one, this should yield
    # the identity matrix.
    wta = np.ones(map.Ny) * (wta * np.pi / 180)
    I = map.ops.tensordotD(
        map.ops.tensordotD(
            np.eye(map.Ny), wta, np.array(1.0), np.array(np.inf), np.array(0.0)
        ),
        -wta,
        np.array(1.0),
        np.array(np.inf),
        np.array(0.0),
    )

    # Compute the mean difference between the diagonal and unity for each l
    x = np.abs(1 - np.diag(I))
    mu = np.array([np.mean(x[l ** 2 : (l + 1) ** 2]) for l in range(map.ydeg + 1)])

    return mu

### Error as a function of spherical harmonic degree

In [ ]:
Nalpha = 0.1
wta = 360 * Nalpha
error10 = get_error(ydeg=10, wta=wta)
error20 = get_error(ydeg=20, wta=wta)
error30 = get_error(ydeg=30, wta=wta)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(error10, color="C0", label=r"$l = 10$")
plt.plot(error20, color="C1", label=r"$l = 20$")
plt.plot(error30, color="C2", label=r"$l = 30$")
plt.grid()
plt.yscale("log")
plt.gca().set_yticks([1e-12, 1e-9, 1e-6, 1e-3, 1e0])
plt.legend(loc="lower right")
plt.title(r"$N \alpha = {:.2f}$".format(Nalpha))
plt.xlabel("spherical harmonic degree")
plt.ylabel(r"relative error");

### Error as a function of $N\alpha$

In [ ]:
ydeg = 20
Nalpha = np.linspace(0, 1, 25)
error = np.zeros((len(Nalpha), ydeg + 1))
for i, Nalpha_i in tqdm(enumerate(Nalpha), total=len(Nalpha)):
    error[i] = get_error(ydeg=ydeg, wta=360 * Nalpha_i)

In [ ]:
plt.figure(figsize=(12, 6))
logerror = np.log10(error)
plt.imshow(
    logerror, extent=(0, ydeg, 0, 1), origin="lower", aspect="auto", vmin=-12, vmax=0
)
cbar = plt.colorbar()
cbar.set_ticks([-12, -9, -6, -3, 0])
cbar.set_ticklabels(
    [r"$10^{-12}$", r"$10^{-9}$", r"$10^{-6}$", r"$10^{-3}$", r"$10^{0}$"]
)
cbar.set_label("relative error")
cont = plt.contour(
    np.arange(ydeg + 1),
    Nalpha,
    logerror,
    [-9, -6, -3, -2, -1],
    colors="w",
    linestyles="solid",
)
fmt = {}
strs = ["1 ppb", "1 ppm", "1 ppt", "1%", "10%"]
for l, s in zip(cont.levels, strs):
    fmt[l] = s
plt.clabel(cont, cont.levels, inline=True, fmt=fmt, fontsize=10)
plt.xlabel("spherical harmonic degree")
plt.ylabel(r"$N \alpha$")
plt.gca().set_xticks([0, 5, 10, 15, 20])
plt.gca().set_xticklabels(["0", "5", "10", "15", "20"]);

### Example

In [ ]:
theta = 90
ydeg = 20

map = starry.Map(ydeg)
map.load("earth", sigma=0.08)
map.amp = 1.0
map[ydeg - 5 :, :] = 0

# Original image
img0 = map.render(projection="rect")

# Differentially rotate it
map[:, :] = map.ops.tensordotD(
    map.y.reshape(1, -1),
    np.array(theta * np.pi / 180),
    np.array(1.0),
    np.array(np.inf),
    np.array(0.0),
)
img1 = map.render(projection="rect")

# Undo the operation
map[:, :] = map.ops.tensordotD(
    map.y.reshape(1, -1),
    np.array(-theta * np.pi / 180),
    np.array(1.0),
    np.array(np.inf),
    np.array(0.0),
)
img2 = map.render(projection="rect")


fig, ax = plt.subplots(2, 2, figsize=(12, 7))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
ax = ax.flatten()
for axis in ax:
    axis.set_xticks([])
    axis.set_yticks([])

im = ax[0].imshow(
    img0, origin="lower", extent=(-180, 180, -90, 90), cmap="plasma", vmin=0, vmax=1
)
cax = make_axes_locatable(ax[0]).append_axes("right", size="5%", pad=0.05)
cax.axis("off")
ax[0].set_title("original")

im = ax[1].imshow(
    img1, origin="lower", extent=(-180, 180, -90, 90), cmap="plasma", vmin=0, vmax=1
)
cax = make_axes_locatable(ax[1]).append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, ax=ax[1], cax=cax, shrink=1)
ax[1].set_title("transformed")

im = ax[2].imshow(
    img2, origin="lower", extent=(-180, 180, -90, 90), cmap="plasma", vmin=0, vmax=1
)
cax = make_axes_locatable(ax[2]).append_axes("right", size="5%", pad=0.05)
cax.axis("off")
ax[2].set_title("reconstructed")

im = ax[3].imshow(
    img2 - img0,
    origin="lower",
    extent=(-180, 180, -90, 90),
    cmap="RdBu",
    vmin=-0.05,
    vmax=0.05,
)
cax = make_axes_locatable(ax[3]).append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, ax=ax[3], cax=cax, shrink=1)
cbar.set_ticks([-0.05, -0.025, 0, 0.025, 0.05])
ax[3].set_title("relative error")

## Timing tests

In [ ]:
def get_flux_num(img, lat, lon, theta, alpha, R):
    # Loop through the timeseries
    flux_num = np.zeros_like(theta)
    for k in range(len(theta)):

        # Compute the longitude, relative to the sub-observer point
        new_lon = np.empty_like(lon)
        if alpha == 0:
            new_lon = lon + theta[k]
            new_lon = ((new_lon + np.pi) % (2 * np.pi)) - np.pi
        else:
            for i, lat_i in enumerate(lat):
                new_lon[i] = lon[i] + theta[k] * np.pi / 180 * (
                    1 - alpha * np.sin(lat_i) ** 2
                )
                new_lon[i] = ((new_lon[i] + np.pi) % (2 * np.pi)) - np.pi

        # Convert to Cartesian in the equatorial frame
        y = np.sin(lat.flat)
        x = np.cos(lat.flat) * np.sin(new_lon.flat)
        z = np.cos(lat.flat) * np.cos(new_lon.flat)

        # Rotate to the observer frame
        y, z = R.dot(np.vstack((y, z)))

        # Sum up observer-facing pixels, weighted by
        # the cosine of the viewing angle (= z)
        flux_num[k] = np.sum(z[z > 0] * img.flat[z > 0]) * (2 * np.pi / len(z[z > 0]))

    return flux_num

In [ ]:
ydeg = 20
npts = 500
res_num = 300
ncalls = 10

theta = np.linspace(0, 360.0, npts)

t0 = np.zeros(ydeg + 1) * np.nan
tD = np.zeros(ydeg + 1) * np.nan
tN0 = np.zeros(ydeg + 1) * np.nan
tND = np.zeros(ydeg + 1) * np.nan
for d in tqdm(range(2, ydeg + 1)):
    map = starry.Map(ydeg=d, inc=75.0)
    map.flux()  # force compile

    # Standard
    map.alpha = 0.0
    tstart = time.time()
    for k in range(ncalls):
        flux_starry_0 = map.flux(theta=theta)
    t0[d] = (time.time() - tstart) / ncalls / npts

    # Differentially rotated
    map.alpha = 1.0
    tstart = time.time()
    for k in range(ncalls):
        map.flux(theta=theta)
    tD[d] = (time.time() - tstart) / ncalls / npts

    # Numerically (standard)
    img = map.render(projection="moll", res=res_num)
    lat, lon = map.get_latlon_grid(projection="moll", res=res_num)
    lat *= np.pi / 180
    lon *= np.pi / 180
    ang = (90 - map.inc) * np.pi / 180
    R = np.array([[np.cos(ang), -np.sin(ang)], [np.sin(ang), np.cos(ang)]])
    tstart = time.time()
    flux_num_0 = get_flux_num(img, lat, lon, theta, 0.0, R)
    tN0[d] = (time.time() - tstart) / npts

    # Numerically (differentially rotated)
    img = map.render(projection="moll", res=res_num)
    lat, lon = map.get_latlon_grid(projection="moll", res=res_num)
    lat *= np.pi / 180
    lon *= np.pi / 180
    ang = (90 - map.inc) * np.pi / 180
    R = np.array([[np.cos(ang), -np.sin(ang)], [np.sin(ang), np.cos(ang)]])
    tstart = time.time()
    get_flux_num(img, lat, lon, theta, 1.0, R)
    tND[d] = (time.time() - tstart) / npts

In [ ]:
error = np.std(flux_starry_0 - flux_num_0) * 1e6
print("Average numerical error: {:.0f} ppm".format(error))

In [ ]:
plt.figure(figsize=(12, 5))

plt.plot(t0, "C0-", label=r"$\mathsf{starry}$: solid")
plt.plot(t0, "C0.")
plt.plot(tD, color="C4", label=r"$\mathsf{starry}$: differential")
plt.plot(tD, "C4.")
plt.plot(tN0, color="C1", label="num: solid")
plt.plot(tN0, "C1.")
plt.plot(tND, color="C3", label="num: differential")
plt.plot(tND, "C3.")

l = np.arange(5, ydeg + 1)
plt.plot(l, 1.75e-6 + 1.5e-9 * l ** 3, "k", lw=3, ls="--", alpha=0.25, label=r"$l^3$")
plt.plot(l, 2e-6 + 1e-9 * l ** 4, "k", lw=3, ls=":", alpha=0.25, label=r"$l^4$")

plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
plt.yscale("log")
plt.gca().set_xticks([0, 5, 10, 15, 20])
plt.gca().set_xticklabels(["0", "5", "10", "15", "20"])
plt.ylabel("time [s]", fontsize=16)
plt.xlabel("spherical harmonic degree", fontsize=16);

## Light curves

### With `starry`

In [ ]:
# Generate a random isotropic l=20 map up to l=15
ydeg_max = 20
ydeg_tru = 15

map = starry.Map(ydeg_max)
power = 5e-3
np.random.seed(3)
for l in range(1, ydeg_tru + 1):
    map[l, :] = np.random.randn(2 * l + 1) * np.sqrt(power / (2 * l + 1))
map.show(projection="moll", colorbar=True)

In [ ]:
ls = range(ydeg_tru + 1)
plt.plot(ls, [np.var(map[l, :]) for l in ls], label="empirical")
plt.plot(ls[1:], [power / (2 * l + 1) for l in ls[1:]], label="specified")
plt.legend()
plt.xlabel("spherical harmonic degree")
plt.ylabel("power");

In [ ]:
prot = 1.0
alpha = 0.02
inc = 75

map.alpha = alpha
map.inc = inc

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(12, 6))
ax = ax.flatten()
for i, t in enumerate(np.linspace(-10, 10, len(ax))):
    map.show(ax=ax[i], projection="moll", res=res, theta=360.0 / prot * t)

In [ ]:
t = np.linspace(-10.0, 10.0, 1000)
theta = 360.0 / prot * t
flux_starry = map.flux(theta=theta)

In [ ]:
plt.plot(t, flux_starry)
plt.xlabel("time [rotations]")
plt.ylabel("flux [arbitrary units]");

### Numerically

In [ ]:
def get_flux_num(map, theta, res=999):

    # Render the image at theta=0
    img = map.render(projection="moll", res=res)
    lat, lon = map.get_latlon_grid(projection="moll", res=res)

    # Convert everything to radians
    theta = np.array(theta) * np.pi / 180
    lat *= np.pi / 180
    lon *= np.pi / 180

    # y-z rotation matrix to observer frame
    ang = (90 - map.inc) * np.pi / 180
    R = np.array([[np.cos(ang), -np.sin(ang)], [np.sin(ang), np.cos(ang)]])

    # Loop through the timeseries
    flux_num = np.zeros_like(theta)
    for k in tqdm(range(len(theta))):

        # Compute the longitude, relative to the sub-observer point
        new_lon = np.empty_like(lon)
        for i, lat_i in enumerate(lat):
            new_lon[i] = lon[i] + theta[k] * (1 - map.alpha * np.sin(lat_i) ** 2)
            new_lon[i] = ((new_lon[i] + np.pi) % (2 * np.pi)) - np.pi

        # Convert to Cartesian in the equatorial frame
        y = np.sin(lat.flat)
        x = np.cos(lat.flat) * np.sin(new_lon.flat)
        z = np.cos(lat.flat) * np.cos(new_lon.flat)

        # Rotate to the observer frame
        y, z = R.dot(np.vstack((y, z)))

        # Sum up observer-facing pixels, weighted by
        # the cosine of the viewing angle (= z)
        flux_num[k] = np.sum(z[z > 0] * img.flat[z > 0]) * (2 * np.pi / len(z[z > 0]))

    return flux_num

In [ ]:
map.alpha = 0.0
theta_0 = np.linspace(0, 360.0, 100)
flux_starry_0 = map.flux(theta=theta_0)
flux_num_0 = get_flux_num(map, theta_0)

In [ ]:
fig, ax = plt.subplots(2, figsize=(12, 6))
ax[0].plot(theta_0, flux_starry_0, lw=3, label="starry")
ax[0].plot(theta_0, flux_num_0, lw=2, label="numerical")
ax[0].legend()
ax[0].set_ylabel("flux [arbitrary units]")

diff = (flux_starry_0 - flux_num_0) * 1e6
ax[1].plot(theta_0, diff, "k.", ms=3)
ax[1].axhline(np.mean(diff), ls="-", lw=1, alpha=0.5)
ax[1].axhline(np.mean(diff) + np.std(diff), ls="--", lw=1, alpha=0.5)
ax[1].axhline(np.mean(diff) - np.std(diff), ls="--", lw=1, alpha=0.5)
ax[1].set_xlabel("angle of rotation")
ax[1].set_ylabel("residuals (ppm)");

In [ ]:
map.alpha = alpha
t = np.linspace(-10.0, 10.0, 1000)
theta = 360.0 / prot * t
flux_starry = map.flux(theta=theta)
flux_num = get_flux_num(map, theta)

In [ ]:
fig, ax = plt.subplots(2, figsize=(12, 6))
ax[0].plot(t, flux_starry, lw=3, label="starry")
ax[0].plot(t, flux_num, lw=2, label="numerical")
ax[0].legend()
ax[0].set_ylabel("flux [arbitrary units]")

diff = (flux_starry - flux_num) * 1e6
ax[1].plot(t, diff, "k.", ms=3)
ax[1].axhline(np.mean(diff), ls="-", lw=1, alpha=0.5)
ax[1].axhline(np.mean(diff) + np.std(diff), ls="--", lw=1, alpha=0.5)
ax[1].axhline(np.mean(diff) - np.std(diff), ls="--", lw=1, alpha=0.5)
ax[1].set_xlabel("time [rotations]")
ax[1].set_ylabel("residuals (ppm)");

In [ ]:
dt = 50
std = np.array([np.std(diff[i : i + dt]) for i in range(len(t) - dt)])
plt.plot(t[:-dt], std, label="estimated")
plt.plot(t[:-dt], std - np.min(std), "k", lw=1, label="corrected")
plt.fill_between(t[:-dt], std - np.min(std), std, alpha=0.05)
plt.legend()
plt.xlabel("time [rotations]")
plt.ylabel("error [ppm]");

### Inference

In [ ]:
np.random.seed(0)
ferr = 1e-4
flux0 = map.flux(theta=theta)
flux = flux0 + ferr * np.random.randn(len(flux0))

In [ ]:
plt.plot(t, flux, "k.", ms=5, alpha=0.5)
plt.plot(t, flux0, "C0", lw=1)
plt.xlabel("time [rotations]")
plt.ylabel("flux [arbitrary units]");

In [ ]:
map = starry.Map(ydeg_tru)
map.inc = 75

In [ ]:
L = np.concatenate(
    [np.ones(2 * l + 1) * power / (2 * l + 1) for l in range(ydeg_tru + 1)]
)

In [ ]:
map.set_data(flux, C=ferr ** 2)

In [ ]:
map.set_prior(L=L)

In [ ]:
alpha_arr = np.linspace(0.01, 0.03, 300)
lnlike = np.zeros_like(alpha_arr)
for i, alpha in tqdm(enumerate(alpha_arr), total=len(alpha_arr)):
    map.alpha = alpha
    lnlike[i] = map.lnlike(theta=theta)

In [ ]:
like = np.exp(lnlike - np.max(lnlike))

In [ ]:
plt.plot(alpha_arr, like)
plt.axvline(0.02, color="C1")
plt.xlabel("differential shear")
plt.ylabel("likelihood");